<a href="https://colab.research.google.com/github/seoyen1122/solar_rag/blob/main/mmlu_pro/astute_rag_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

문서 분석 로직 & 외부 문서는 그냥 불러오기


In [1]:
pip install langchain-upstage langchain-community pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.3/84.3 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.8/295.8 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 60.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 1.9 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.22.1
    Uninstalling tokenizers-0.22.1:
      Successfully uninstalled tokenizers-0.22.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviou

In [2]:
pip install wikipedia-api

  Preparing metadata (setup.py) ... done
  Created wheel for wikipedia-api: filename=Wikipedia_API-0.8.1-py3-none-any.whl size=15383 sha256=cd018f7495e7b9a44bce1d38008f21edfe97e0fa8973dc1ca49e4dd49cc5aefe
  Stored in directory: /root/.cache/pip/wheels/33/3c/79/b36253689d838af4a0539782853ac3cc38a83a6591ad570dde
Successfully built wikipedia-api


In [3]:
pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.6/23.6 MB 82.9 MB/s eta 0:00:00


In [4]:
!pip install sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 19.6 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.20.3
    Uninstalling tokenizers-0.20.3:
      Successfully uninstalled tokenizers-0.20.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-upstage 0.7.5 requires tokenizers<0.21.0,>=0.20.0, but you have tokenizers 0.22.1 which is incompatible.


In [5]:
import os
import json
from typing import List, Tuple

from langchain_upstage import UpstageEmbeddings, ChatUpstage
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import ChatPromptTemplate

import wikipediaapi
import pandas as pd

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
cd /content/drive/MyDrive/nlp team project

/content/drive/MyDrive/nlp team project


In [8]:
UPSTAGE_API_KEY = "up_VYzFNHEoEJPfAwYUNp5v9n1CPnMOm"

In [ ]:
df = pd.read_csv("testset.csv")

In [ ]:
llm_classifier = ChatUpstage(
    api_key=UPSTAGE_API_KEY,
    model="solar-pro2",
    temperature=0.1, #거의 항상 같은 결과, 안정적이게
)

In [9]:
llm_solver = ChatUpstage(
    api_key=UPSTAGE_API_KEY,
    model="solar-pro2",
    temperature=0.1,
)

In [10]:
llm_solar_pro = ChatUpstage(
    api_key=UPSTAGE_API_KEY,
    model="solar-pro2",
    temperature=0.1, #거의 항상 같은 결과, 안정적이게
)

In [11]:
category_index = {
    "law":        "/content/drive/MyDrive/nlp team project/code/mmlu_category/law",
    "psychology": "/content/drive/MyDrive/nlp team project/code/mmlu_category/psychology",
    "business":   "/content/drive/MyDrive/nlp team project/code/mmlu_category/business",
    "philosophy": "/content/drive/MyDrive/nlp team project/code/mmlu_category/philosophy",
    "history":    "/content/drive/MyDrive/nlp team project/code/mmlu_category/history",
}

categories = list(category_index.keys())

In [12]:

emb = UpstageEmbeddings(api_key=UPSTAGE_API_KEY, model="solar-embedding-1-large-passage")

# 카테고리별 faiss 한번에 로드해서 캐시
vectorstore = {}
for cat, path in category_index.items():
    vectorstore[cat] = FAISS.load_local(
        folder_path=path,
        embeddings=emb,
        allow_dangerous_deserialization=True,
    )

wiki = wikipediaapi.Wikipedia(user_agent= "NLP-RAG/1.0", language = 'en')

**1st LLM**:

- return category

- return 3 keywords

In [13]:
from langchain_core.prompts import ChatPromptTemplate
import json

# 질문 분석을 위한 전용 프롬프트
P_ANALYZE_TMPL = """
You are an expert Question Analyst for MMLU-Pro.
Your task is NOT to answer the question, but to dissect it so that a researcher can find the best evidence.

[Available Categories]
- law
- psychology
- business
- philosophy
- history
- other

[Task]
Analyze the question and options to provide a structured JSON output.
You must differentiate between search strategies for **Textbooks** (Vector DB) and **Wikipedia** (Keyword Match).

[Output Fields]
1. "category": Choose ONE from the list above.
2. "core_intent": Summarize what the question is asking in 1 sentence.
3. "constraints": List strict conditions (time period, specific person, negation 'NOT', location).
4. "search_queries": Generate 3-5 **sentence-style** queries for vector search.
   - **CRITICAL**: Translate specific scenarios into **Professional/Legal/Academic Terminology**.
   - Example: "Man hit neighbor" -> "Tort law battery elements"
5. "wiki_keywords": Extract 3-5 **specific entities or nouns** for Wikipedia title search.
   - Example: ["Battery (tort)", "Common law", "Trespass"]

[Example Shot]
Question: "According to Jean Piaget, at which stage of cognitive development do children begin to think logically about abstract concepts and hypothetical situations?"
Options: (A) Sensorimotor (B) Preoperational (C) Concrete operational (D) Formal operational

Output:
{{
  "category": "psychology",
  "core_intent": "Identify the Piagetian stage associated with abstract and hypothetical reasoning.",
  "constraints": ["Jean Piaget's theory", "Abstract concepts", "Hypothetical situations", "Logic"],
  "search_queries": [
    "Piaget cognitive development stages abstract reasoning",
    "Characteristics of formal operational stage",
    "Difference between concrete and formal operational stages"
  ],
  "wiki_keywords": [
    "Jean Piaget",
    "Cognitive development",
    "Formal operational stage",
    "Theory of cognitive development"
  ]
}}

[Real Input]
Question:
{question_text}

Options:
{options_text}

Return ONLY the valid JSON object.
""".strip()

p_analyze_prompt = ChatPromptTemplate.from_template(P_ANALYZE_TMPL)

def run_p_analyze(question_text: str, options: dict, llm=None) -> dict:
    """
    질문을 분석하여 검색 쿼리와 제약 조건을 추출합니다.
    """
    if llm is None:
        llm = llm_solver  # 기존에 정의된 llm 객체 사용

    # 옵션 텍스트화
    options_str = "\n".join([f"({k}) {v}" for k, v in options.items()])

    messages = p_analyze_prompt.format_messages(
        question_text=question_text,
        options_text=options_str
    )

    resp = llm.invoke(messages)
    raw = resp.content.strip()

    # JSON 파싱 (실패 시 기본값 반환 처리 포함)
    try:
        # 마크다운 코드 블록 제거 등 기본적인 정제
        if "```json" in raw:
            raw = raw.split("```json")[1].split("```")[0].strip()
        elif "```" in raw:
            raw = raw.split("```")[0].strip()

        analysis = json.loads(raw)
        return analysis
    except Exception as e:
        print(f"[WARN] p_analyze failed: {e}")
        # 실패 시 기본값: 검색어는 질문 그대로
        return {
            "core_intent": "Unknown",
            "constraints": [],
            "search_queries": [question_text]
        }

In [14]:
def fetch_wiki_context(keywords, window_size=2000):
    snippets = []
    seen_titles = set()

    for kw in keywords:
        kw = (kw or "").strip()
        if not kw: continue

        candidates = [kw, kw.title()]
        # 괄호 제거 등 추가 처리
        if "(" in kw: candidates.append(kw.split("(")[0].strip())

        found_flag = False
        for cand in candidates:
            if cand in seen_titles: continue

            try:
                page = wiki.page(cand)
                if page.exists():
                    seen_titles.add(cand)
                    full_text = page.text

                    # [개선] 단순히 앞부분만 자르는 게 아니라, 키워드가 등장하는 위치를 찾음
                    # 키워드가 텍스트 내에 있으면 그 주변을 가져옴. 없으면 앞부분 가져옴.
                    lower_text = full_text.lower()
                    lower_kw = kw.lower()
                    start_idx = lower_text.find(lower_kw)

                    if start_idx == -1:
                        # 키워드 못 찾으면 그냥 앞부분
                        display_text = full_text[:window_size]
                    else:
                        # 키워드 주변부 (앞으로 500자, 뒤로 1500자 정도)
                        start_pos = max(0, start_idx - 500)
                        end_pos = min(len(full_text), start_idx + window_size)
                        display_text = full_text[start_pos:end_pos]

                    clean_text = display_text.replace('\n', ' ')
                    snippets.append(f"[Wikipedia: {page.title}]\n...{clean_text}...")
                    found_flag = True
                    break
            except Exception:
                continue

    if not snippets:
        return ""

    return "\n\n".join(snippets)

In [15]:
import re
import json
from typing import List, Dict, Any

import pandas as pd
from langchain_core.prompts import ChatPromptTemplate

############################################
# 0. 상수 & 기본 설정
############################################

TOP_K_TEXTBOOK = 5
TOP_K_WIKI = 5

In [16]:
############################################
# 1. 질문 + 옵션(A~I) 파싱
############################################

# 1. 질문 + 옵션(A~J) 파싱
def parse_question_and_options(prompt: str):
    """
    prompt: QUESTION + (A) ~ (J) 옵션이 한 문자열에 들어있는 형태
    return: question_text(str), options(dict: 'A'~'J' -> str)
    """
    pattern = r"\(([A-J])\)\s*"
    matches = list(re.finditer(pattern, prompt))

    if not matches:
        return prompt.strip(), {}

    first = matches[0]
    question_text = prompt[:first.start()].strip()

    options: Dict[str, str] = {}
    for idx, m in enumerate(matches):
        letter = m.group(1)  # 'A'~'J'
        start = m.end()
        end = matches[idx + 1].start() if idx + 1 < len(matches) else len(prompt)
        opt_text = prompt[start:end].strip()
        options[letter] = opt_text

    return question_text, options


In [17]:
############################################
# 2. "Final Answer: X" 파서
############################################

def extract_final_answer_letter(text: str) -> str:
    """
    p_ans 출력에서 'Final Answer: X' 의 X(A~J)를 뽑기.
    """
    m = re.search(r"Final Answer:\s*([A-J])\s*$", text, flags=re.IGNORECASE | re.MULTILINE)
    if m:
        return m.group(1).upper()
    return ""

In [18]:

############################################
# 3. Memory p_gen (INTERNAL 문서 생성)
############################################

P_MEM_TMPL = """
You are a knowledgeable expert in {category}.

Task: Using ONLY your own internal knowledge (without seeing any external documents),
generate a short document that provides accurate and relevant information to help
answer the given exam question.

If you truly do not know the answer or lack enough information, explicitly state
"I don't know" rather than hallucinating facts.

Exam Question:
{question_text}

Write ONE coherent document that:
- Focuses only on concepts, definitions, and relations that are relevant to the question.
- Is self-contained and clear enough to help another model answer the question.
- Avoids unnecessary details.

Document:
""".strip()

p_mem_prompt = ChatPromptTemplate.from_template(P_MEM_TMPL)


def run_p_mem(category: str, question_text: str, llm=None) -> str:
    if llm is None:
        llm = llm_solver
    msgs = p_mem_prompt.format_messages(
        category=category,
        question_text=question_text,
    )
    resp = llm.invoke(msgs)
    return resp.content.strip()

In [19]:
def run_p_gen_for_docs(
    category: str,             # 사용 안 함 (호환성 위해 남겨둠)
    source_type: str,          # "TEXTBOOK" or "WIKIPEDIA"
    question_text: str,        # 사용 안 함 (호환성 위해 남겨둠)
    raw_passages: List[str],   # 검색된 원본 텍스트 리스트
    start_doc_idx: int = 1,
    llm=None,                  # 사용 안 함 (호환성 위해 남겨둠)
) -> Tuple[str, int]:
    """
    [수정됨] LLM을 통한 요약/재작성을 수행하지 않습니다.
    검색된 Raw Passage를 그대로 포맷팅하여 반환합니다.
    이렇게 해야 정보 손실(Information Loss) 없이 p_con이나 p_ans 단계로 넘어갑니다.
    """
    doc_blocks = []
    cur_idx = start_doc_idx

    for k, passage in enumerate(raw_passages, start=1):
        # 빈 텍스트 무시
        if not passage or not passage.strip():
            continue

        # [중요] LLM 호출(invoke) 없이 원본 텍스트를 그대로 넣습니다.
        # 필요하다면 너무 긴 문단만 파이썬 문자열 슬라이싱으로 자르세요 (예: [:2000])
        clean_passage = passage.strip()

        block = (
            f"[Doc {cur_idx} | SOURCE=EXTERNAL({source_type}) | ORIG_ID={source_type}_{k}]\n"
            f"{clean_passage}"
        )
        doc_blocks.append(block)
        cur_idx += 1

    return "\n\n".join(doc_blocks), cur_idx

In [20]:
!pip install rank_bm25

In [21]:
import numpy as np
from typing import List
from rank_bm25 import BM25Okapi

############################################
# 5. Retrieval helpers (Hybrid: Vector + BM25)
############################################

# 전역 캐시 (BM25 인덱스를 매번 만들지 않기 위해 저장)
bm25_store = {}   # {category: BM25Okapi_object}
doc_store = {}    # {category: [text_list]}

def init_bm25_for_category(category: str):
    """
    [초기화] 해당 카테고리의 모든 문서를 로드하여 BM25 인덱스를 생성
    """
    if category in bm25_store:
        return

    if category not in vectorstore:
        return

    print(f"[Init] Building BM25 index for: {category}...")
    vs = vectorstore[category]

    try:
        # FAISS 메모리에서 모든 문서 텍스트 추출
        # (주의: FAISS 로드 시 데이터가 메모리에 있어야 함)
        all_docs = list(vs.docstore._dict.values())
        texts = [d.page_content for d in all_docs]

        if not texts:
            print(f"[Warn] No texts found in docstore for {category}.")
            return

        # 토크나이징 (단순 띄어쓰기 기준)
        tokenized_corpus = [doc.lower().split() for doc in texts]

        # 저장
        bm25_store[category] = BM25Okapi(tokenized_corpus)
        doc_store[category] = texts
        print(f"[Init] BM25 ready for {category} ({len(texts)} docs)")

    except Exception as e:
        print(f"[Warn] Failed to init BM25 for {category} (Pure Vector mode will be used): {e}")


def reciprocal_rank_fusion(results_list: List[List[str]], k=60) -> List[str]:
    """
    [RRF 알고리즘] 여러 검색 결과의 순위를 합산하여 재정렬
    """
    fused_scores = {}

    for docs in results_list:
        for rank, doc_text in enumerate(docs):
            if doc_text not in fused_scores:
                fused_scores[doc_text] = 0
            # 순위가 높을수록(rank가 작을수록) 높은 점수 부여
            fused_scores[doc_text] += 1 / (rank + k)

    # 점수 높은 순 정렬
    reranked = sorted(fused_scores.items(), key=lambda x: x[1], reverse=True)
    return [text for text, score in reranked]


def get_textbook_passages(category: str, query: str) -> List[str]:
    """
    [수정된 함수] Hybrid Search 적용
    1. Vector Search (의미 검색) -> 10개
    2. BM25 Search (키워드 검색) -> 10개
    3. RRF Fusion -> 상위 5개 반환
    """
    if category not in vectorstore:
        return []

    # 0. BM25 준비
    if category not in bm25_store:
        init_bm25_for_category(category)

    # 1. Vector Search (Semantic)
    vs = vectorstore[category]
    # 후보를 10개 정도 가져옴
    vec_candidates = vs.similarity_search(query, k=10)
    vec_results = [d.page_content for d in vec_candidates]

    # 2. BM25 Search (Keyword) - 만약 준비 안됐으면 생략
    bm25_results = []
    if category in bm25_store:
        bm25 = bm25_store[category]
        docs = doc_store[category]

        tokenized_query = query.lower().split()
        # 키워드 매칭 상위 10개
        bm25_results = bm25.get_top_n(tokenized_query, docs, n=10)

    # 3. Fusion (하나만 있으면 그것만 반환)
    if not bm25_results:
        final_passages = vec_results
    else:
        # 두 결과를 섞음
        final_passages = reciprocal_rank_fusion([vec_results, bm25_results], k=60)

    # 최종 TOP_K 개수만큼 자르기
    return final_passages[:TOP_K_TEXTBOOK]


def get_wiki_passages(keywords: List[str]) -> List[str]:
    """
    (기존 유지)
    """
    try:
        raw = fetch_wiki_context(keywords)
    except Exception as e:
        print(f"[WARN] Wikipedia retrieval failed: {e}")
        return []

    raw = (raw or "").strip()
    if not raw:
        return []
    return [raw]

In [22]:
############################################
# 6. Iterative Knowledge Consolidation p_con (Few-Shot Optimized)
############################################

P_CON_TMPL = """
You are a Knowledge Integrator for MMLU-Pro dataset.

Task: Consolidate information from both your own memorized documents (INTERNAL)
and externally retrieved documents (EXTERNAL) in response to the given exam question.

Guidelines:
* For documents that provide consistent information, cluster them together and
  summarize the key details into a single, concise document.
* For documents with conflicting information, separate them into distinct documents,
  ensuring each document captures the unique perspective or data.
* Exclude any information that is irrelevant to the question.
* For each NEW document you create, clearly indicate:
  - Whether the source was INTERNAL(MEMORY) or EXTERNAL(TEXTBOOK/WIKIPEDIA).
  - The original document numbers that contributed to it (e.g., FROM=1,3,4).

[Examples]

<Example 1: Conflict Resolution>
Initial Context:
[Doc 1 | SOURCE=INTERNAL(MEMORY)]
The Treaty of Versailles was signed in 1918.
[Doc 2 | SOURCE=EXTERNAL(TEXTBOOK)]
The Treaty of Versailles was signed on June 28, 1919, officially ending WWI.

Question: When was the Treaty of Versailles signed?

New Context:
[Doc 1 | SOURCE=EXTERNAL(TEXTBOOK) | FROM=2]
The Treaty of Versailles was signed on June 28, 1919.
(Correction: Internal memory incorrectly stated 1918, which was the armistice year, not the treaty signing.)
</Example 1>

<Example 2: Merging Consistent Info>
Initial Context:
[Doc 1 | SOURCE=EXTERNAL(WIKIPEDIA)]
Classical conditioning involves a neutral stimulus becoming a conditioned stimulus.
[Doc 2 | SOURCE=EXTERNAL(TEXTBOOK)]
Pavlov's dog experiment demonstrated classical conditioning by pairing a bell with food.

Question: Explain the mechanism of classical conditioning.

New Context:
[Doc 1 | SOURCE=MERGED(EXTERNAL) | FROM=1,2]
Classical conditioning is a learning process where a neutral stimulus (like a bell) becomes a conditioned stimulus after being paired with an unconditioned stimulus (like food). This was demonstrated in Pavlov's dog experiment.
</Example 2>

[Real Task]

Initial Context (numbered documents):
{context_init}

Last Context (may be empty):
{last_context}

Question:
{question_text}

Now produce your New Context following the examples.
""".strip()

p_con_prompt = ChatPromptTemplate.from_template(P_CON_TMPL)

def run_p_con(
    question_text: str,
    context_init: str,
    last_context: str = "",
    llm=None,
) -> str:
    if llm is None:
        try:
            llm = llm_solar_pro
        except NameError:
            llm = llm_solver

    msgs = p_con_prompt.format_messages(
        question_text=question_text,
        context_init=context_init,
        last_context=last_context,
    )
    resp = llm.invoke(msgs)
    return resp.content.strip()


In [23]:
############################################
# 7. Knowledge Consolidation + Answer Finalization p_ans (10-Option Optimized)
############################################

import re

def extract_final_answer_letter(text: str) -> str:
    """
    모델이 내뱉는 다양한 형식을 모두 잡아내는 강력한 추출 함수
    """
    if not text: return ""

    # 1순위: "Final Answer" 뒤에 오는 첫 번째 영문자 (A-J) 찾기
    # 예: "**Final Answer:** (A)", "Final Answer: Option A", "Final Answer is [A]"
    # [^\nA-J]* : 줄바꿈이나 A-J가 나오기 전까지의 모든 특수문자(:, *, 공백 등)를 무시
    match = re.search(r"Final Answer[^\nA-J]*([A-J])", text, flags=re.IGNORECASE | re.MULTILINE)
    if match:
        return match.group(1).upper()

    # 2순위: <ANSWER> 태그 (프롬프트에서 지시한 경우)
    match_tag = re.search(r"<ANSWER>\s*\(?([A-J])\)?", text, flags=re.IGNORECASE)
    if match_tag:
        return match_tag.group(1).upper()

    # 3순위: 최후의 수단 - 문장 맨 끝에 있는 (A) 형태 찾기
    fallback = re.findall(r"\(?([A-J])\)?", text.split("\n")[-1])
    if fallback:
        return fallback[-1].upper()

    return "" # 정말 답이 없는 경우


In [24]:
CATEGORY_META = {
    "law": {
        "role": (
            "You are a distinguished Law Professor and Bar Exam Grader. "
            "You specialize in applying strict legal doctrines to complex fact patterns. "
            "Your expertise covers Contracts, Torts, Criminal Law, Property, Constitutional Law, and Evidence."
        ),
        "guidelines": (
            "- Spot the precise legal issue (e.g., specific crime, tort, contract doctrine).\n"
            "- Apply traditional common-law elements unless a statute in the context clearly overrides them.\n"
            "- For crimes, require that ALL elements are satisfied; if one element is missing, that crime is wrong.\n"
            "- For 'best defense' or 'most likely' questions, prefer options that directly attack missing elements or raise strong procedural bars "
            "(e.g., Statute of Frauds, Statute of Limitations)."
        ),
    },
    "business": {
        "role": (
            "You are a CPA (Certified Public Accountant) and Economics Professor. "
            "You excel at financial accounting, managerial accounting, corporate finance, and micro/macroeconomics."
        ),
        "guidelines": (
            "- Carefully extract all numerical data and relationships before computing.\n"
            "- Show each calculation step explicitly (no guessing): check signs, totals, and units.\n"
            "- For finance questions, check if time value of money (PV/FV, discounting) is implied by dates or interest rates.\n"
            "- For conceptual items, ground answers in standard models (supply/demand, elasticity, basic game theory) rather than intuition."
        ),
    },
    "history": {
        "role": (
            "You are an expert Historian specializing in primary source analysis. "
            "You handle World, U.S., European, and Asian history with a focus on chronology and causality."
        ),
        "guidelines": (
            "- First fix the time period and region using names, events, or terminology.\n"
            "- Eliminate options that are anachronistic (wrong century, wrong ruler, wrong war).\n"
            "- Distinguish short-term triggers from long-term structural causes when evaluating explanations.\n"
            "- When reading passages, consider author, audience, and purpose to match the most plausible interpretation."
        ),
    },
    "philosophy": {
        "role": (
            "You are an expert Philosopher specializing in Ethics, Metaphysics, and Epistemology. "
            "You are precise with terminology and familiar with canonical arguments (Kant, Mill, Aristotle, etc.)."
        ),
        "guidelines": (
            "- Identify the relevant school or author first (e.g., Kantian deontology vs. Millian utilitarianism).\n"
            "- Pay close attention to technical terms (e.g., validity vs. truth, necessary vs. sufficient, a priori vs. a posteriori).\n"
            "- Reject options that contradict the core commitments of the view (e.g., utilitarianism ignoring consequences).\n"
            "- Prefer options that match the exact formulation of the principle, not just something that sounds reasonable."
        ),
    },
    "psychology": {
        "role": (
            "You are a Clinical and Research Psychologist. "
            "You specialize in DSM-5 criteria, developmental stages, cognitive psychology, and experimental design."
        ),
        "guidelines": (
            "- For diagnosis, match symptoms strictly to DSM-5 criteria, especially duration, severity, and exclusion conditions.\n"
            "- For development/theory items, map behaviors to the correct named theory and precise stage (e.g., Piaget, Kohlberg, Erikson).\n"
            "- In research design questions, clearly identify IV, DV, and likely confounds; distinguish correlation from causation.\n"
            "- For ethics scenarios, prioritize APA principles such as informed consent, confidentiality, and minimizing harm."
        ),
    },
}

DEFAULT_CATEGORY_META = {
    "role": (
        "You are an expert exam solver for MMLU-Pro. You are careful, analytical, and precise."
    ),
    "guidelines": (
        "- Read the QUESTION carefully and respect all stated constraints.\n"
        "- Use the provided Context as primary evidence and avoid unsupported assumptions.\n"
        "- Eliminate options that conflict with the facts, definitions, or chronology in the Context.\n"
        "- Choose the single best-supported option, even if other options seem partially plausible."
    ),
}

In [25]:
P_ANS_TMPL = """
You are an expert multiple-choice exam solver for MMLU-Pro, using an Astute-RAG style reasoning process.

[Role]
{category_role}

[Category-Specific Guidelines]
{category_guidelines}

You are given:
[Initial Context]  (short, noisy, or partial)
{context_init}

[Consolidated Context]  (higher-quality, filtered evidence)
{context_con}

[QUESTION]
{question_with_options}

Your job is to carefully combine your INTERNAL knowledge with the given EXTERNAL context,
and then follow the **three-stage Astute-RAG reasoning** below.

--------------------------------
Stage 1 – Evidence Aggregation (Astute Step 1)
--------------------------------
1. From the QUESTION ONLY (ignore context for now), extract 3–5 key clues:
   - important concepts, time periods, entities, relationships, or definitions.
2. Using the CONTEXT (Initial + Consolidated), list 3–7 **relevant evidence statements**:
   - each statement should be short (1 sentence),
   - include only facts that help discriminate between the options.
3. If the context contradicts itself, give higher trust to the **Consolidated Context**,
   but never violate explicit constraints stated in the QUESTION.

--------------------------------
Stage 2 – Option-wise Diagnosis with Score & Confidence (Astute Step 2)
--------------------------------
For each option (A, B, C, D, ...), do the following:

1. In 1–2 sentences, compare the option with the key clues + evidence:
   - Explain how it is supported or contradicted.
   - Check whether it satisfies all critical constraints in the QUESTION.

2. Assign three evaluations to this option:
   - A **Label**: one of
     * "CLEARLY SUPPORTED"
     * "PARTIALLY SUPPORTED / DUBIOUS"
     * "CLEARLY CONTRADICTED"
   - A numerical **Score** between 0 and 1:
     * 0.0 ≈ completely wrong, 1.0 ≈ very strongly supported.
   - A short **Confidence** phrase: one of ["high", "medium", "low"].

Use the following format for each option:

Option A:
- Reasoning: ...
- Label: CLEARLY SUPPORTED
- Score: 0.82
- Confidence: high

(Repeat this exact structure for options B, C, D, ...)

--------------------------------
Stage 3 – Global Consistency Check & Final Decision (Astute Step 3)
--------------------------------
1. Look at ALL options, their Labels, and Scores together.
2. Eliminate every option that is:
   - labeled "CLEARLY CONTRADICTED", or
   - has Score < 0.30 (unless every option is very low-scoring).
3. Among the remaining options, choose **one single BEST** answer by asking:
   - Which option is most strongly and directly supported by the evidence and key clues?
   - Does any option rely on extra assumptions not justified by the context?
4. If two options have similar Scores, briefly explain why the higher one
   is still better given the QUESTION wording and constraints.

Finally, on the last line of your response, output the answer in the format:

Final Answer: X

(where X is a single capital letter such as A, B, C, D, ...).
Do NOT output anything after this line.
""".strip()
p_ans_prompt = ChatPromptTemplate.from_template(P_ANS_TMPL)

In [26]:
# 살짝 변경
def run_p_ans(
    context_init: str,
    context_con: str,
    full_question: str,
    category: str,
    llm=None,
) -> Dict[str, Any]:
    """
    p_gen/p_con으로 만들어진 context를 기반으로,
    astute-RAG 방식(INTERNAL vs EXTERNAL, Step 1~4)을 그대로 따르되,
    category별 역할/전략 텍스트를 추가하여 특화된 solver로 동작하게 함.
    """
    if llm is None:
        try:
            llm = llm_solar_pro
        except NameError:
            llm = llm_solver

    meta = CATEGORY_META.get(category, DEFAULT_CATEGORY_META)

    msgs = p_ans_prompt.format_messages(
        category_role=meta["role"],
        category_guidelines=meta["guidelines"],
        context_init=context_init,
        context_con=context_con,
        question_with_options=full_question,
    )
    resp = llm.invoke(msgs)
    raw = resp.content.strip()
    letter = extract_final_answer_letter(raw)
    return {"final_answer": letter, "raw_reasoning": raw}

In [27]:
def solve_mmlu_astute_style(
    full_prompt: str,
    use_wiki: bool = True,
    n_vote: int = 5
) -> Dict[str, Any]:

    # ---------------------------------------------------------
    # 1. 통합 분석 (One-Shot Prompt 적용된 run_p_analyze 사용)
    # ---------------------------------------------------------
    question_text, options = parse_question_and_options(full_prompt)

    try:
        # [변경] 여기서 Category, Textbook Query, Wiki Keywords를 한 번에 다 가져옴
        analysis = run_p_analyze(question_text, options, llm=llm_solver)
    except NameError:
        print("[Error] run_p_analyze not defined. Using fallback.")
        analysis = {}

    # 결과 추출 (안전하게 get 사용)
    # 1) 카테고리 (구버전 함수 classify_... 삭제됨)
    category = analysis.get("category", "business")

    # 2) 제약조건
    constraints = analysis.get("constraints", [])

    # 3) 검색어 분리
    # 교과서용 (문장형)
    textbook_queries = analysis.get("search_queries", [question_text])
    # 위키용 (명사형)
    wiki_keywords = analysis.get("wiki_keywords", [])

    # [Fallback] 만약 위키 키워드가 비었으면 교과서 쿼리라도 씀
    if not wiki_keywords:
        wiki_keywords = textbook_queries

    print(f"   -> [Analysis] Category: {category}")
    print(f"   -> [Textbook Query] {textbook_queries[:1]}...")
    print(f"   -> [Wiki Keywords] {wiki_keywords}")

    # 벡터 DB 확인
    if category not in vectorstore:
        print(f"   -> [Warn] Category '{category}' not found. Fallback to 'business'.")
        category = "business"

    # ---------------------------------------------------------
    # 2. 검색 (Dual-Track: Hybrid Vector + Wiki)
    # ---------------------------------------------------------

    # Track A: Textbook (Vector + BM25 Hybrid)
    # 문장형 쿼리에 제약조건을 더해서 문맥 강화
    combined_textbook_query = " ".join(textbook_queries) + " " + " ".join(constraints)
    tb_passages = get_textbook_passages(category, combined_textbook_query)

    # Track B: Wikipedia (Entity Search)
    # 명사형 키워드로 정확한 표제어 매칭
    wiki_passages = []
    if use_wiki:
        wiki_passages = get_wiki_passages(wiki_keywords)

    # ---------------------------------------------------------
    # 3. Context 생성 (p_mem -> p_gen -> p_con)
    # ---------------------------------------------------------
    doc_blocks = []
    next_doc_idx = 1

    # Internal Memory
    mem_doc_text = run_p_mem(category, question_text)
    if mem_doc_text.strip():
        doc_blocks.append(f"[Doc {next_doc_idx} | SOURCE=INTERNAL(MEMORY)]\n{mem_doc_text}")
        next_doc_idx += 1

    # Textbook Docs (LLM 요약 없이 Pass-through)
    if tb_passages:
        tb_block, next_doc_idx = run_p_gen_for_docs(
            category, "TEXTBOOK", question_text, tb_passages, next_doc_idx, None
        )
        if tb_block.strip(): doc_blocks.append(tb_block)

    # Wiki Docs (LLM 요약 없이 Pass-through)
    if wiki_passages:
        wiki_block, next_doc_idx = run_p_gen_for_docs(
            category, "WIKIPEDIA", question_text, wiki_passages, next_doc_idx, None
        )
        if wiki_block.strip(): doc_blocks.append(wiki_block)

    context_init = "\n\n".join(doc_blocks)

    # Consolidation
    context_con = run_p_con(question_text=question_text, context_init=context_init)

    # ---------------------------------------------------------
    # 4. Voting (Self-Consistency)
    # ---------------------------------------------------------

    # Voting용 질문 구성 (제약조건 명시)
    constraints_str = "\n".join([f"- {c}" for c in constraints])
    augmented_question = f"{full_prompt}\n\n[CRITICAL CONSTRAINTS]\n{constraints_str}"

    # 다양성을 위한 Temperature 설정
    llm_voter = ChatUpstage(api_key=UPSTAGE_API_KEY, model="solar-pro2", temperature=0.7)

    votes = []
    reasonings = []

    print(f"   -> [Voting] Running {n_vote} iterations...")
    for i in range(n_vote):
        try:
            ans = run_p_ans(
                context_init=context_init,
                context_con=context_con,
                full_question=augmented_question,
                category=category,
                llm=llm_voter
            )
            letter = ans["final_answer"]

            # 유효한 답만 투표
            if letter and letter in "ABCDEFGHIJ":
                votes.append(letter)

            reasonings.append(f"[Run {i+1} Answer: {letter}]\n{ans['raw_reasoning']}")

        except Exception as e:
            print(f"      [Vote Error]: {e}")
            continue

    # 최종 결과 집계
    if not votes:
        final_ans = "A" # Fallback
        vote_summary = "None"
    else:
        # 최빈값 선택
        count = Counter(votes)
        final_ans = count.most_common(1)[0][0]
        vote_summary = str(dict(count))
        print(f"   -> [Result] Winner: {final_ans} | Votes: {vote_summary}")

    full_log = "\n" + "="*40 + "\n".join(reasonings)

    return {
        "final_answer": final_ans,
        "category": category,
        "analysis": analysis,
        "keywords": wiki_keywords, # 변경됨: extracted_keywords -> wiki_keywords
        "context_init": context_init,
        "context_con": context_con,
        "raw_reasoning": full_log,
        "vote_summary": vote_summary
    }

100개

In [ ]:
import pandas as pd

df = pd.read_csv("/content/testset_100.csv")
QUESTION_COL = "prompts"

results = []
for i, row in df.iterrows():
    q = row[QUESTION_COL]
    print(f"[{i}] Solving: {q[:60]}...")
    out = solve_mmlu_astute_style(q)
    results.append(out["final_answer"])
    df.loc[i, "pred_category"] = out["category"]
    df.loc[i, "kw1"] = out["keywords"][0] if len(out["keywords"]) > 0 else ""
    df.loc[i, "kw2"] = out["keywords"][1] if len(out["keywords"]) > 1 else ""
    df.loc[i, "kw3"] = out["keywords"][2] if len(out["keywords"]) > 2 else ""
    df.loc[i, "rag_cot_answer"] = out["final_answer"]
    df.loc[i, "cot_full"] = out["raw_reasoning"]

df.to_csv("astute-rag-1126-final.csv", index=False)
print("저장 완료:1126.csv")

[0] Solving: QUESTION4868) In 1797, John Frere made a discovery that he d...
   -> [Analysis] Category: history
   -> [Textbook Query] ['John Frere 1797 archaeological discovery description']...
   -> [Wiki Keywords] ['John Frere', 'Hoxne Palaeolithic site', 'Acheulean handaxe', 'Paleoanthropology history', 'Deep stratigraphy archaeology']
   -> [Voting] Running 5 iterations...
   -> [Result] Winner: I | Votes: {'I': 5}
[1] Solving: QUESTION586) BobGafneyand Susan Medina invested $40,000 and ...
   -> [Analysis] Category: business
   -> [Textbook Query] ['Partnership accounting income distribution with interest and management fee']...
   -> [Wiki Keywords] ['Partnership accounting', 'Income distribution agreement', 'Capital contribution accounting', 'Partnership profit sharing', 'Managerial compensation in partnerships']
   -> [Voting] Running 5 iterations...
   -> [Result] Winner: F | Votes: {'F': 5}
[2] Solving: QUESTION138) If at the beginning of each month a deposit of ...
   -> [A

In [ ]:
import pandas as pd

# 1. 파일 경로
GT_PATH = "/content/testset_100.csv"        # 정답 파일
PRED_PATH = "/content/drive/MyDrive/nlp team project/astute-rag-1126-final.csv"       # 출력 파일

# 2. 컬럼 이름
GT_COL = "answers"               # testset.csv에서 정답 컬럼
PRED_COL = "rag_cot_answer"      # baseline.csv에서 예측 컬럼

# 3. csv 로드
gt_df = pd.read_csv(GT_PATH)
pred_df = pd.read_csv(PRED_PATH)

# 4. Series 추출
gt = gt_df[GT_COL].astype(str)
pred = pred_df[PRED_COL].astype(str)

# 5. 정규화 함수
def normalize_choice(x: str) -> str:
    x = x.strip().upper()
    for ch in x:
        if "A" <= ch <= "Z":
            return ch
    return x

# 인덱스 리셋이 포인트
gt_norm = gt.apply(normalize_choice).reset_index(drop=True)
pred_norm = pred.apply(normalize_choice).reset_index(drop=True)

print(len(gt_norm), len(pred_norm))  # 둘 다 25 나오는지 체크 한번 해보고

# 6. 정확도 계산
correct = (gt_norm == pred_norm)
accuracy = correct.mean()

print(f"총 문제 수: {len(gt_norm)}")
print(f"정답 개수: {correct.sum()}")
print(f"정확도: {accuracy * 100:.2f}%")

기존 testset.csv

In [29]:
import pandas as pd
import traceback
from collections import Counter

# 1. 데이터 로드 (원본 데이터 보존을 위해 copy 사용)
# 테스트할 범위 슬라이싱 (예: 25번부터 끝까지)
input_file = "testset.csv"
df = pd.read_csv(input_file)[25:].copy()
QUESTION_COL = "prompts"

print(f"🚀 Processing {len(df)} questions from {input_file}...")

# 2. 실행 루프
results = []

for i, row in df.iterrows():
    q = row[QUESTION_COL]
    print(f"\n[{i}] Solving: {q[:60]}...")

    try:
        # 메인 솔버 호출
        out = solve_mmlu_astute_style(full_prompt=q, use_wiki=True, n_vote=5)

        # 결과 저장
        final_ans = out.get("final_answer", "A") # 기본값 A
        category = out.get("category", "business")
        keywords = out.get("keywords", [])
        reasoning = out.get("raw_reasoning", "")

        results.append(final_ans)

        # DataFrame 업데이트
        df.loc[i, "rag_cot_answer"] = final_ans
        df.loc[i, "pred_category"] = category
        df.loc[i, "cot_full"] = reasoning

        # 키워드 저장 (개수가 가변적이므로 안전하게 처리)
        df.loc[i, "kw1"] = keywords[0] if len(keywords) > 0 else ""
        df.loc[i, "kw2"] = keywords[1] if len(keywords) > 1 else ""
        df.loc[i, "kw3"] = keywords[2] if len(keywords) > 2 else ""

        # (선택) 전체 키워드와 쿼리도 저장해두면 분석할 때 좋습니다
        df.loc[i, "all_keywords"] = str(keywords)

        # 중간 저장 (선택 사항: 코랩 런타임 끊김 대비)
        if i % 10 == 0:
            df.to_csv("temp_checkpoint.csv", index=False)

    except Exception as e:
        print(f"💥 Error processing index {i}: {e}")
        traceback.print_exc()
        # 에러 시 기본값 채우기 (멈추지 않도록)
        df.loc[i, "rag_cot_answer"] = "A"
        df.loc[i, "cot_full"] = f"Error: {str(e)}"

# 3. 최종 저장
output_file = "astute_rag_final_수정.csv"
df.to_csv(output_file, index=False)
print(f"\n✅ 모든 작업 완료! 결과 저장됨: {output_file}")

🚀 Processing 25 questions from testset.csv...

[25] Solving: QUESTION26) QUESTION 6) A psychologist is asked to see a 10-...
   -> [Analysis] Category: psychology
   -> [Textbook Query] ['Ethical guidelines for minor consent in school-based psychological counseling']...
   -> [Wiki Keywords] ['Informed consent', 'Minor consent in therapy', 'APA Ethics Code', 'Child assent in research', 'School psychology ethics']
[Init] Building BM25 index for: psychology...
[Init] BM25 ready for psychology (2232 docs)
   -> [Voting] Running 5 iterations...
   -> [Result] Winner: D | Votes: {'D': 5}

[26] Solving: QUESTION27) A man is at home in his apartment, alone, late a...
   -> [Analysis] Category: law
   -> [Textbook Query] ['Common law elements of robbery burglary and trespassing']...
   -> [Wiki Keywords] ['Robbery (law)', 'Burglary', 'Trespassing', 'Aggravated assault', 'Common law crimes']
[Init] Building BM25 index for: law...
[Init] BM25 ready for law (91859 docs)
   -> [Voting] Running 5 i

In [30]:
import pandas as pd

# 1. 파일 경로
GT_PATH = "testset.csv"          # 정답 파일
PRED_PATH = "astute_rag_final_수정.csv"  # 출력 파일 (현재 경로에 저장됨)

# 2. 컬럼 이름
GT_COL = "answers"               # testset.csv에서 정답 컬럼
PRED_COL = "rag_cot_answer"      # baseline.csv에서 예측 컬럼

# 3. csv 로드
gt_df = pd.read_csv(GT_PATH)[25:]
pred_df = pd.read_csv(PRED_PATH)

# 4. Series 추출
gt = gt_df[GT_COL].astype(str)
pred = pred_df[PRED_COL].astype(str)

# 5. 정규화 함수
def normalize_choice(x: str) -> str:
    x = x.strip().upper()
    for ch in x:
        if "A" <= ch <= "Z":
            return ch
    return x

# 인덱스 리셋 (중요: 길이를 맞추기 위해)
gt_norm = gt.apply(normalize_choice).reset_index(drop=True)
pred_norm = pred.apply(normalize_choice).reset_index(drop=True)

print(f"Ground Truth 개수: {len(gt_norm)}, Prediction 개수: {len(pred_norm)}")

# 6. 정확도 계산
correct = (gt_norm == pred_norm)
accuracy = correct.mean()

print(f"총 문제 수: {len(gt_norm)}")
print(f"정답 개수: {correct.sum()}")
print(f"정확도: {accuracy * 100:.2f}%")

Ground Truth 개수: 25, Prediction 개수: 25
총 문제 수: 25
정답 개수: 20
정확도: 80.00%
